In [1]:
import os
os.chdir('..')

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

# https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/
# export TAVILY_API_KEY='xxx'
tavily = TavilySearchResults(max_results=5)

# 这里定义了3个工具，但这里3个工具其实都调用了同一个方法
access_tools = {
    'tavily1': dict(
        func=tavily,
        desc='这是一个专门用来搜索音乐的搜索引擎',
        args=[dict(
            name='input',
            description='搜索关键词',
            type='string'
        )]
    ),
    'tavily2': dict(
        func=tavily,
        desc='这是一个专门用来搜索烹饪美食的搜索引擎',
        args=[dict(
            name='input',
            description='搜索关键词',
            type='string'
        )]
    ),
    'tavily3': dict(
        func=tavily,
        desc='这是一个搜索引擎，如果实在找不到合适的搜素工具可以使用这个工具进行搜索',
        args=[dict(
            name='input',
            description='搜索关键词',
            type='string'
        )]
    ),
}

In [3]:
os.environ['LANG'] = 'ch'
from templates.make_prompts import *

def make_prompt(query, history=[], first_query=True, **kwargs):
    if first_query:
        query = make_query(query)
    else:
        query = make_thought_chain_query(query, **kwargs)

    prompt = make_skeleton(
        query=query,
        additional_prompts=[
            make_today_date(),
            make_location('广州'),
            make_encourage(),
            make_access_tools(access_tools),
            make_chat_history(history) if history else '',
            make_thought_chain(list(access_tools.keys())),
        ]
    )
    return prompt

In [4]:
from models.bailian import LLM

# export API_KEY='xxx'
llm = LLM(api_key=os.environ.get('API_KEY'))

In [5]:
import re

def parse_thought_chain(content):
    thought = re.search('Thought: (.+)', content).group(1)
    action = re.search('Action: (.+)', content).group(1)
    action_input = re.search('Action Input: (.+)', content).group(1)

    return thought, action, action_input

In [6]:
import json

def parse_action_input(action_input):
    try:
        action_input = json.loads(action_input)
    except:
        action_input = {'input': action_input}
    return action_input

In [7]:
history = []

def run(query):
    chat_chain = []
    
    prompt = make_prompt(query, history=history)
    chat_chain.append(prompt)
    
    content = llm(prompt)
    chat_chain.append(content)

    while 'Final Answer' not in content:
        thought, action, action_input = parse_thought_chain(content)
        observation = access_tools[action]['func'].invoke(**parse_action_input(action_input))
    
        prompt = make_prompt(
            query, 
            history=history,
            first_query=False,
            thought=thought,
            action=action,
            action_input=action_input,
            observation=observation,
        )
        chat_chain.append(prompt)
    
        content = llm(prompt)
        chat_chain.append(content)

    answer = re.search('Final Answer: (.+)', content).group(1)
    history.append((query, answer))

    return answer, chat_chain

In [8]:
query = '今天的天气怎么样'
answer, chat_chain = run(query)
print('-------\n'.join(chat_chain))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



今天是2024-05-24。现在在广州。请尽你所能去回答问题。
你可以使用下面列举的工具:
tavily1: 这是一个专门用来搜索音乐的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily2: 这是一个专门用来搜索烹饪美食的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily3: 这是一个搜索引擎，如果实在找不到合适的搜素工具可以使用这个工具进行搜索 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]

请一步一步地使用以下的格式进行思考:

Question: 你必须回答的问题
Thought: 你必须思考你应该怎么做
Action: 你下一步采取的行动，你可以使用 [tavily1, tavily2, tavily3] 提供的工具
Action Input: 你下一步行动中使用的工具的输入
Observation: 你下一步行动的结果返回
... (Thought/Action/Action Input/Observation 这些操作步骤可以重复多次)
Thought: 我现在知道怎么回答这个问题了
Final Answer: 上述的问题的最终的答案是

开始!

Question: 今天的天气怎么样

-------
Thought: 我需要查询广州今天的天气，但是提供的工具中没有直接查询天气的。我将尝试使用tavily3这个通用搜索引擎来查找相关信息。
Action: tavily3
Action Input: {"input": "广州 2024-05-24 天气"}
-------

今天是2024-05-24。现在在广州。请尽你所能去回答问题。
你可以使用下面列举的工具:
tavily1: 这是一个专门用来搜索音乐的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily2: 这是一个专门用来搜索烹饪美食的搜索引擎 s.args: [{"na

In [9]:
query = '需要带伞吗'
answer, chat_chain = run(query)
print('-------\n'.join(chat_chain))


今天是2024-05-24。现在在广州。请尽你所能去回答问题。
你可以使用下面列举的工具:
tavily1: 这是一个专门用来搜索音乐的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily2: 这是一个专门用来搜索烹饪美食的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily3: 这是一个搜索引擎，如果实在找不到合适的搜素工具可以使用这个工具进行搜索 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]

这是以往的历史信息:

Question: 今天的天气怎么样
Answer: 2024年5月24日，广州的天气预报为雷阵雨，气温约为23℃至29℃之间，风向为无持续风向，风力为微风。请记得带雨具出行。

请一步一步地使用以下的格式进行思考:

Question: 你必须回答的问题
Thought: 你必须思考你应该怎么做
Action: 你下一步采取的行动，你可以使用 [tavily1, tavily2, tavily3] 提供的工具
Action Input: 你下一步行动中使用的工具的输入
Observation: 你下一步行动的结果返回
... (Thought/Action/Action Input/Observation 这些操作步骤可以重复多次)
Thought: 我现在知道怎么回答这个问题了
Final Answer: 上述的问题的最终的答案是

开始!

Question: 需要带伞吗

-------
Thought: 根据之前的天气预报信息，今天广州有雷阵雨，因此可以判断是否需要带伞。
Final Answer: 根据今天的天气预报显示有雷阵雨，建议外出时携带雨伞以备不时之需。


In [10]:
query = '列10首适合今天天气听的周杰伦的歌曲'
answer, chat_chain = run(query)
print('-------\n'.join(chat_chain))


今天是2024-05-24。现在在广州。请尽你所能去回答问题。
你可以使用下面列举的工具:
tavily1: 这是一个专门用来搜索音乐的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily2: 这是一个专门用来搜索烹饪美食的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily3: 这是一个搜索引擎，如果实在找不到合适的搜素工具可以使用这个工具进行搜索 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]

这是以往的历史信息:

Question: 今天的天气怎么样
Answer: 2024年5月24日，广州的天气预报为雷阵雨，气温约为23℃至29℃之间，风向为无持续风向，风力为微风。请记得带雨具出行。

Question: 需要带伞吗
Answer: 根据今天的天气预报显示有雷阵雨，建议外出时携带雨伞以备不时之需。

请一步一步地使用以下的格式进行思考:

Question: 你必须回答的问题
Thought: 你必须思考你应该怎么做
Action: 你下一步采取的行动，你可以使用 [tavily1, tavily2, tavily3] 提供的工具
Action Input: 你下一步行动中使用的工具的输入
Observation: 你下一步行动的结果返回
... (Thought/Action/Action Input/Observation 这些操作步骤可以重复多次)
Thought: 我现在知道怎么回答这个问题了
Final Answer: 上述的问题的最终的答案是

开始!

Question: 列10首适合今天天气听的周杰伦的歌曲

-------
Thought: 我需要找到适合雷阵雨天气听的周杰伦的歌曲，最适合的工具是音乐搜索引擎tavily1。

Action: tavily1

Action Input: {"input": "周杰伦 雷阵雨 天气 适合歌曲"}

-------

今天是2024-05-24。现在在广州。请尽你所能去回答问题。